In [1]:
from copy import deepcopy
from heapq import heappop, heappush
from itertools import product
from math import inf

import numpy as np

In [2]:
with open("input", "r") as f:
    blizzard_map_raw = [line.strip() for line in f.readlines()]
    
blizzard_map = {(i-1, j-1): list() if blizzard == '.' else list(blizzard)
                for i, line in enumerate(blizzard_map_raw)
                for j, blizzard in enumerate(line)
                if blizzard != '#'
               }

size_y, size_x = len(blizzard_map_raw) - 2, len(blizzard_map_raw[0]) - 2
shape = np.array((size_y, size_x))

In [3]:
def blizzard_progress(blizzard_map):
    _blizzard_map = deepcopy(blizzard_map)
    blizzard_moves = {
        '>': (0, 1),
        '<': (0, -1),
        'v': (1, 0),
        '^': (-1, 0)
    }
    
    while True:
        old_blizzard_map = deepcopy(_blizzard_map)
        _blizzard_map = {}
        
        for coords in old_blizzard_map.keys():
            while len(old_blizzard_map[coords]) > 0:
                blizzard = old_blizzard_map[coords].pop()
                _coords = tuple((np.array(coords) + np.array(blizzard_moves[blizzard])) % shape)
                if _coords not in _blizzard_map:
                    _blizzard_map[_coords] = list()
                _blizzard_map[_coords].append(blizzard)

            if coords not in _blizzard_map:
                _blizzard_map[coords] = list()
                
        yield _blizzard_map 

In [4]:
def h(point_a, point_b):
    return sum(abs(coord_a - coord_b)
               for coord_a, coord_b in zip(point_a, point_b))

In [5]:
def reconstruct_path(came_from, current):
    path = [current]

    while current in came_from:
        current = came_from[current]
        path = [current] + path
    
    return path

In [6]:
def path_in_blizzard(start, goal, blizzard_map, h):

    blizzard_maps = [blizzard_map]
    blizzard_step = blizzard_progress(blizzard_map)
    
    came_from = {}
    open_set = []
    g_score = {start: 0}
    f_score = {start: h(start[:2], goal)}
    
    heappush(open_set, (f_score[start], start))
    
    x, y, t = start
    if goal not in blizzard_map:
        return None
    
    while len(open_set) > 0:
        _, current = heappop(open_set)
        x, y, t = current
        
        if (x, y) == goal:
            return reconstruct_path(came_from, current), blizzard_maps[-1]
        
        neighbors = []
        while t + 1 > len(blizzard_maps) - 1:
            blizzard_maps.append(next(blizzard_step))

        neighbors = [(x + x0, y + y0, t + 1) for x0, y0 in product(range(-1, 2), repeat=2)
                     if abs(x0) + abs(y0) <= 1
                     and (((shape - np.array((x + x0, y + y0)) > 0).all()
                           and (np.array((x + x0, y + y0)) >= 0).all())
                          or (x == x + x0 and y == y + y0)
                          or (x + x0, y + y0) == goal)
                     and len(blizzard_maps[t + 1][x + x0, y + y0]) == 0]
        

        
        for neighbor in neighbors:
            tentative_g_score = g_score.get(current, inf) + 1

            if tentative_g_score < g_score.get(neighbor, inf):
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g_score
                f_score[neighbor] = tentative_g_score + h(neighbor, goal)
                if (f_score[neighbor], neighbor) not in open_set:
                    heappush(open_set, (f_score[neighbor], neighbor))
    return None

In [7]:
start = (-1, blizzard_map_raw[0].index('.') - 1)
goal = (len(blizzard_map_raw) - 2, blizzard_map_raw[-1].index('.') - 1)

### Part 1

In [8]:
path, blizzard_map = path_in_blizzard((*start, 0), goal, blizzard_map, h)

len(path) - 1

279

### Part 2

In [9]:
path_back, blizzard_map = path_in_blizzard((*goal, 0), start, blizzard_map, h)
path_forward, blizzard_map = path_in_blizzard((*start, 0), goal, blizzard_map, h)

len(path) - 1 + len(path_forward) - 1 + len(path_back) - 1

762